In [2]:
import pandas as pd
import numpy as np
import PIL
from IPython.display import Image, display
from transformers import AutoImageProcessor, AutoModelForImageClassification, CLIPProcessor, CLIPModel
import tqdm

In [3]:
image_df = pd.read_csv('train_nto.csv')
image_df = image_df.drop(columns='unified_class')
image_df

,class_id,image_name
0,2,0fa5f1cd824ca3705b654ada33437337.JPG
1,3,da2969de9de9ca78eee5ba249e52ce89.JPG
2,4,5b5d9c79cec42ea630d9f29ae59d2d6d.JPG
3,11,e5dfc9f687530959332a2f8aa0594140.JPG
4,6,50de65ad44e8dc71ed7f122eff3f2164.JPG
...,...,...
40153,4,e744521bf5841656a7a2069d16e73739.JPG
40154,19,1ee34ce50ff7d386abf6f03846d46b3c.JPG
40155,4,0023e78f09736e035419568ea1f6e6d4.JPG
40156,2,cb2576cbf98b9d97621fe5298f30dc8a.JPG


In [21]:
sub = pd.read_csv('sub_classif1.csv') # sub_classif1.csv - test csv with valid rows
sub

,image_name,predicted_class
0,09974f39ebf33a94ac1505f8e4352ce3.JPG,17
1,80bb8b892e17f56869e9045880ea42e4.JPG,1
2,ecbe0b8bcbc6f32c0f2fd46c06833b0c.JPG,18
3,ed1f7ed239eda609f56e15484886a02e.JPG,4
4,03b269aaeb40fe866cb41693b3afb0fc.JPG,7
...,...,...
16337,cca8403503d71d11eaf097598a53df20.JPG,14
16338,c1ef97f00759adf20ad85ef201fb7093.JPG,14
16339,3e607f0fd625c1d5b693490520fda08a.JPG,4
16340,a56e0607deb47b2494e9ca85c2f8caf6.JPG,2


In [5]:
clip = CLIPModel.from_pretrained('openai/clip-vit-base-patch32')
preproc = CLIPProcessor.from_pretrained('openai/clip-vit-base-patch32')

In [6]:
day_night = AutoModelForImageClassification.from_pretrained('Woleek/day-night')
preproc_dn = AutoImageProcessor.from_pretrained('Woleek/day-night')

/home/xbar19/ML/MLEnv/lib/python3.10/site-packages/transformers/models/mobilenet_v2/feature_extraction_mobilenet_v2.py:28: FutureWarning: The class MobileNetV2FeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use MobileNetV2ImageProcessor instead.
  warnings.warn(


In [7]:
category = ['forest', 'mountains', 'field']

In [18]:
def get_place_features(paths, subset: str): # subset in ['train', 'test']
    places = []
    for path in tqdm.tqdm(paths):
        with PIL.Image.open(f'{subset}_nto/{path}').convert('RGB') as img0:
            places.append(category[np.argmax(clip(**preproc(images=img0, return_tensors='pt', text=['forest', 'mountains', 'field'], padding=True)).logits_per_image.detach().numpy())])
    return places

In [12]:
def get_day_night_features(paths, subset: str): # subset in ['train', 'test']
    dn_features = []
    for path in tqdm.tqdm(paths):
        with PIL.Image.open(f'{subset}_nto/{path}').convert('RGB') as img0:
            dn_features.append(np.argmax(day_night(**preproc_dn(images=img0, return_tensors='pt')).logits.detach().numpy()))
    return dn_features

In [22]:
image_df1 = image_df.copy()
sub1 = sub.copy()

In [ ]:
train_dn = get_day_night_features(image_df['image_name'].values, 'train')
test_dn = get_day_night_features(sub['image_name'].values, 'test')

In [ ]:
train_place = get_place_features(image_df['image_name'].values, 'train')
test_place = get_place_features(sub['image_name'].values, 'test')

In [ ]:
image_df['is_night'] = train_dn
sub['is_night'] = test_dn

In [ ]:
image_df1['place'] = train_place
sub1['place'] = test_place

In [20]:
# cat_preds = []
# for path in tqdm.tqdm(sub['image_name'].values):
#     with PIL.Image.open(f'test_nto/{path}').convert('RGB') as img0:
#         cat_preds.append(category[np.argmax(clip(**preproc(images=img0, return_tensors='pt', text=['forest', 'mountains', 'field'], padding=True)).logits_per_image.detach().numpy())])

In [14]:
sub['place'].value_counts()

place
forest       11629
field         3590
mountains     1123
Name: count, dtype: int64

In [16]:
sub

,image_name,predicted_class,place
0,09974f39ebf33a94ac1505f8e4352ce3.JPG,17,forest
1,80bb8b892e17f56869e9045880ea42e4.JPG,1,forest
2,ecbe0b8bcbc6f32c0f2fd46c06833b0c.JPG,18,forest
3,ed1f7ed239eda609f56e15484886a02e.JPG,4,field
4,03b269aaeb40fe866cb41693b3afb0fc.JPG,7,forest
...,...,...,...
16337,cca8403503d71d11eaf097598a53df20.JPG,14,field
16338,c1ef97f00759adf20ad85ef201fb7093.JPG,14,field
16339,3e607f0fd625c1d5b693490520fda08a.JPG,4,forest
16340,a56e0607deb47b2494e9ca85c2f8caf6.JPG,2,field


In [ ]:
image_df.to_csv('train_nto_dn.csv')
sub.to_csv('test_nto_dn.csv')

In [10]:
image_df1.to_csv('train_nto_place.csv')

In [15]:
sub1.to_csv('test_nto_place.csv')